In [1]:
import pandas as pd
import numpy as np

In [2]:
# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

In [3]:
data_positive['label'] = 'pos'
len(data_positive)

114911

In [4]:
data_negative['label'] = 'neg'
len(data_negative)

111923

In [5]:
frames = [data_positive, data_negative]
data = pd.concat(frames)

In [6]:
len(data)

226834

In [7]:
data.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",pos
1,"Да, все-таки он немного похож на него. Но мой ...",pos
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,pos
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",pos
4,@irina_dyshkant Вот что значит страшилка :D\nН...,pos


In [8]:
import re

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


df = [preprocess_text(t) for t in data['text']]

In [9]:
data['review'] = df

In [10]:
data.head()

,text,label,review
0,"@first_timee хоть я и школота, но поверь, у на...",pos,USER хоть я и школота но поверь у нас то же са...
1,"Да, все-таки он немного похож на него. Но мой ...",pos,да все таки он немного похож на него но мой ма...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,pos,rt USER ну ты идиотка я испугалась за тебя
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",pos,rt USER кто то в углу сидит и погибает от голо...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,pos,USER вот что значит страшилка d но блин посмот...


In [11]:
df = data.drop('text', axis=1)

In [12]:
len(df)

226834

In [13]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [14]:
blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
print(len(blanks), 'blanks: ', blanks)

0 blanks:  []


In [15]:
df['label'].value_counts()

pos    114911
neg    111923
Name: label, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

X = df['review']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# Naïve Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

In [18]:
text_clf_nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [19]:
# Form a prediction set
predictions = text_clf_nb.predict(X_test)

In [20]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[27887  9211]
 [10335 27423]]


In [21]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.73      0.75      0.74     37098
         pos       0.75      0.73      0.74     37758

    accuracy                           0.74     74856
   macro avg       0.74      0.74      0.74     74856
weighted avg       0.74      0.74      0.74     74856



In [22]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.7388853264935342


In [23]:
text_clf_lsvc.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [24]:
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)

In [25]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[27135  9963]
 [ 9423 28335]]


In [26]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.74      0.73      0.74     37098
         pos       0.74      0.75      0.75     37758

    accuracy                           0.74     74856
   macro avg       0.74      0.74      0.74     74856
weighted avg       0.74      0.74      0.74     74856



In [27]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.7410227637063161


In [28]:
predictions

array(['neg', 'neg', 'neg', ..., 'neg', 'pos', 'pos'], dtype=object)

In [29]:
X_test.head()

100170    мыс на меня обиделась я ей даже ничего не сделала
105843               аааааааааааааааааааа не хочу на работу
60463     у меня какой то особенный вид ушей d некоторые...
98352     USER он неплохой человек в жизни я работала в ...
113055    rt USER домааааа ехали на такси пели песни отд...
Name: review, dtype: object

In [30]:
myreview = 'это очень просторная комната с замечательной люстрой'

In [31]:
print(text_clf_lsvc.predict([myreview]))

['pos']


In [32]:
myreview = 'я не понимаю это хорошо или плохо'

In [33]:
print(text_clf_lsvc.predict([myreview]))

['neg']
